In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import torch
import pickle
from tqdm import tqdm
import gc
from  datetime import datetime as dt

In [2]:
DATAPATH = '/root/autodl-tmp'

In [3]:
# data = pd.read_csv('data/150w_result.csv')

# data = pd.read_csv('data/food_usage_need_training_test_set.csv')

# data = data.sample(frac=1, random_state=2024).reset_index(drop=True)

data = pd.DataFrame()

for fp in tqdm(os.listdir(DATAPATH + '/20M')):
    df_ = pickle.load(open(f'{DATAPATH}/20M/{fp}', 'rb'))
    data = pd.concat([data, df_], axis = 0) 
    del df_
data = data.reset_index()

print(data.shape)

100%|██████████| 20/20 [01:15<00:00,  3.77s/it]


(19940584, 3)


In [4]:
data = data.reset_index(drop = True)[['id', 'content']]

In [5]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [6]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
# model = SentenceTransformer('BAAI/bge-base-zh-v1.5')

# model = SentenceTransformer('BAAI/bge-m3')
# model = SentenceTransformer('BAAI/llm-embedder')
# model = SentenceTransformer('google-bert/bert-base-chinese')
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')
# model = SentenceTransformer('FacebookAI/xlm-roberta-base')
model = SentenceTransformer('Qwen/Qwen2-0.5B')
# model = SentenceTransformer('openai-community/gpt2')

sum([np.prod(p.size()) for p in model.parameters()])

/root/miniconda3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-20 16:29:16.199608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 16:29:17.580609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-20 16:29:17.580774: W tensorflow/compiler/xla/

494032768

In [7]:
# model.save(f'{DATAPATH}/pretrained_models/Qwen2-0.5B')

In [8]:
print(model.get_max_seq_length())
print(model.get_sentence_embedding_dimension())

32768
896


In [9]:
model.tokenizer.vocab_size

151643

In [10]:
a = data['content'].values[30]
print("Raw:\n", a)

tokens = model.tokenizer.encode(a)
print()
reserve_tokens = model.tokenizer.decode(tokens)
print("tokenizer:\n",reserve_tokens.replace(' ',''))

Raw:
 Day1 今早体重89.1
在2022年12月30日这一天体重下降到88.1！开心，完成了一年的计划。2023年希望自己依然可以自律。
早饭吃到了🍩➕桃酥➕🍞
午饭一碗🍚配了很多的菜
晚饭又又吃了很多的小吃
2023年的第一天很开心，希望新的一年开心，平安，万事顺意[心] #减肥#

tokenizer:
 Day1今早体重89.1
在2022年12月30日这一天体重下降到88.1！开心，完成了一年的计划。2023年希望自己依然可以自律。
早饭吃到了🍩➕桃酥➕🍞
午饭一碗🍚配了很多的菜
晚饭又又吃了很多的小吃
2023年的第一天很开心，希望新的一年开心，平安，万事顺意[心]#减肥#


In [11]:
import json 

CHUNK_SIZE = 10000
BATCH_SIZE = 200
OUTPUT_DIR = f'{DATAPATH}/qwen2_05B_embeddings'

os.makedirs(OUTPUT_DIR, exist_ok=True)

embeding_idx = []
for filename in os.listdir(OUTPUT_DIR):
  with open(f'{OUTPUT_DIR}/{filename}', 'rb') as f:
    embeding_idx += list((pickle.load(f)).keys())

data_rest = data[~data['id'].isin(embeding_idx)]

print("dropped data size: ", data[data['id'].isin(embeding_idx)].shape)
print("remaining data size: ", data_rest.shape)
del embeding_idx

data_rest['batch'] = [int(i/CHUNK_SIZE) for i in range(len(data_rest))]

print('cuda: ', torch.cuda.is_available())
torch.cuda.empty_cache()
gc.collect()
for b, d in tqdm(data_rest.groupby('batch')):
  timestamp = dt.now().strftime('%Y-%m-%d_%H:%M:%S')
  with torch.no_grad():
    embeddings = model.encode(d['content'].tolist(), show_progress_bar=True, batch_size=BATCH_SIZE, device='cuda')
    embeddings_dict = dict(zip(d['id'].tolist(), embeddings.tolist()))
    #pickle.dump(embeddings_dict, open(os.path.join(OUTPUT_DIR, f'batch_{timestamp}.pkl'), 'wb'))
    with open(f'{OUTPUT_DIR}/batch_{timestamp}.pickle', 'wb') as f :
        pickle.dump(embeddings_dict, f)
  del embeddings
  torch.cuda.empty_cache()
  gc.collect()

dropped data size:  (0, 2)
remaining data size:  (19940584, 2)
cuda:  True


  0%|          | 0/1995 [00:00<?, ?it/s]

Batches:   0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/1995 [02:23<?, ?it/s]


TypeError: write() argument must be str, not bytes

In [12]:
with open(f'{OUTPUT_DIR}/batch_{timestamp}.pickle', 'wb') as f :
        pickle.dump(embeddings_dict, f)